In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
data=pd.read_csv("train.csv")
data

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233149,626432,63213,105405,60.72,34,20700,48,1050,01-08-88,Salaried,...,0,0,4084,0,0,0,1yrs 9mon,3yrs 3mon,0,0
233150,606141,73651,100600,74.95,34,23775,51,990,05-12-88,Self employed,...,0,0,1565,0,0,0,0yrs 6mon,0yrs 6mon,0,0
233151,613658,33484,71212,48.45,77,22186,86,2299,01-06-76,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
233152,548084,34259,73286,49.10,77,22186,86,2299,26-03-94,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0


In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

In [6]:
data['u_Employment.Type']=labelencoder.fit_transform(data['Employment.Type'])
u_Employment_Type=data[['Employment.Type','u_Employment.Type']].drop_duplicates().sort_values(by='u_Employment.Type')
u_Employment_Type

,Employment.Type,u_Employment.Type
0,Salaried,0
1,Self employed,1
87,NaN,2


In [7]:
data['u_PERFORM_CNS.SCORE.DESCRIPTION']=labelencoder.fit_transform(data['PERFORM_CNS.SCORE.DESCRIPTION'])
u_PERFORM_CNS_SCORE_DESCRIPTION=data[['u_PERFORM_CNS.SCORE.DESCRIPTION','PERFORM_CNS.SCORE.DESCRIPTION']].drop_duplicates().sort_values(by='u_PERFORM_CNS.SCORE.DESCRIPTION')
u_PERFORM_CNS_SCORE_DESCRIPTION

,u_PERFORM_CNS.SCORE.DESCRIPTION,PERFORM_CNS.SCORE.DESCRIPTION
5,0,A-Very Low Risk
11,1,B-Very Low Risk
13,2,C-Very Low Risk
8,3,D-Very Low Risk
17,4,E-Low Risk
36,5,F-Low Risk
152,6,G-Low Risk
19,7,H-Medium Risk
1,8,I-Medium Risk
164,9,J-High Risk


In [8]:
def convert_age(age_str):
    years, months = age_str.split(' ')
    years = int(years[:-3]) if 'yrs' in years else 0
    months = int(months[:-3]) if 'mon' in months else 0
    return '{:.2f}'.format(years*12 + months)

In [9]:
data['u_CREDIT.HISTORY.LENGTH'] = data['CREDIT.HISTORY.LENGTH'].apply(convert_age)

In [10]:
data['u_CREDIT.HISTORY.LENGTH']=data['u_CREDIT.HISTORY.LENGTH'].astype(float)

In [11]:
data['Date.of.Birth']=pd.to_datetime(data['Date.of.Birth'],infer_datetime_format=True)

In [12]:
data['AVERAGE.ACCT.AGE'] = data['AVERAGE.ACCT.AGE'].apply(convert_age)

In [13]:
data['AVERAGE.ACCT.AGE']=data['AVERAGE.ACCT.AGE'].astype(float)

In [14]:
data['DisbursalDate']=pd.to_datetime(data['DisbursalDate'],infer_datetime_format=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 44 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   UniqueID                             233154 non-null  int64         
 1   disbursed_amount                     233154 non-null  int64         
 2   asset_cost                           233154 non-null  int64         
 3   ltv                                  233154 non-null  float64       
 4   branch_id                            233154 non-null  int64         
 5   supplier_id                          233154 non-null  int64         
 6   manufacturer_id                      233154 non-null  int64         
 7   Current_pincode_ID                   233154 non-null  int64         
 8   Date.of.Birth                        233154 non-null  datetime64[ns]
 9   Employment.Type                      225493 non-null  object        
 

In [16]:
data1=data.drop(['Employment.Type','PERFORM_CNS.SCORE.DESCRIPTION','CREDIT.HISTORY.LENGTH'],axis=1)

In [17]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   UniqueID                             233154 non-null  int64         
 1   disbursed_amount                     233154 non-null  int64         
 2   asset_cost                           233154 non-null  int64         
 3   ltv                                  233154 non-null  float64       
 4   branch_id                            233154 non-null  int64         
 5   supplier_id                          233154 non-null  int64         
 6   manufacturer_id                      233154 non-null  int64         
 7   Current_pincode_ID                   233154 non-null  int64         
 8   Date.of.Birth                        233154 non-null  datetime64[ns]
 9   DisbursalDate                        233154 non-null  datetime64[ns]
 

In [18]:
data1=data1.set_index('UniqueID')

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
y = data1['loan_default']
x = data1.drop(columns={'asset_cost','loan_default',
                        'Date.of.Birth',
                        'DisbursalDate',
                        'SEC.DISBURSED.AMOUNT',
                        'disbursed_amount',
                        'manufacturer_id',
                        'MobileNo_Avl_Flag',
                        'VoterID_flag',
                        'u_PERFORM_CNS.SCORE.DESCRIPTION',
                        'PRI.SANCTIONED.AMOUNT','supplier_id','ltv','SEC.CURRENT.BALANCE'}, axis=1)

vif_data=pd.DataFrame()
vif_data["features"]=x.columns
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]
print(vif_data)



                               features       VIF
0                             branch_id  2.146828
1                    Current_pincode_ID  2.925745
2                              State_ID  3.399421
3                      Employee_code_ID  3.369716
4                           Aadhar_flag  3.941804
5                              PAN_flag  1.117234
6                          Driving_flag  1.066401
7                         Passport_flag  1.005785
8                     PERFORM_CNS.SCORE  2.844980
9                       PRI.NO.OF.ACCTS  3.219353
10                     PRI.ACTIVE.ACCTS  5.793597
11                    PRI.OVERDUE.ACCTS  1.718195
12                  PRI.CURRENT.BALANCE  1.575615
13                 PRI.DISBURSED.AMOUNT  1.296863
14                      SEC.NO.OF.ACCTS  3.474468
15                     SEC.ACTIVE.ACCTS  3.756697
16                    SEC.OVERDUE.ACCTS  1.436026
17                SEC.SANCTIONED.AMOUNT  1.266417
18                   PRIMARY.INSTAL.AMT  1.099049


In [20]:
data1=data1.drop(['Date.of.Birth',
                        'DisbursalDate',
                        'SEC.DISBURSED.AMOUNT',
                        'disbursed_amount',
                        'manufacturer_id',
                        'MobileNo_Avl_Flag',
                        'VoterID_flag',
                        'u_PERFORM_CNS.SCORE.DESCRIPTION',
                        'PRI.SANCTIONED.AMOUNT','supplier_id','ltv','SEC.CURRENT.BALANCE'],axis=1)

In [21]:
data1.shape

(233154, 28)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
X=data1.drop(['loan_default'],axis=1)
X.shape

(233154, 27)

In [24]:
y=data1['loan_default']
y.value_counts()

0    182543
1     50611
Name: loan_default, dtype: int64

In [25]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [26]:
y_resampled.shape

(365086,)

In [27]:
X_train, X_test, y_train, y_test=train_test_split(X_resampled, y_resampled, test_size=0.80, random_state=42) 

In [28]:
logreg =  LogisticRegression(solver='liblinear') 

In [29]:
logreg.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

In [30]:
y_pred=logreg.predict(X_test) 

In [31]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [32]:
from sklearn import metrics 
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

cnf_matrix 

array([[66401, 79412],
       [58462, 87794]])

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.46      0.49    145813
           1       0.53      0.60      0.56    146256

    accuracy                           0.53    292069
   macro avg       0.53      0.53      0.53    292069
weighted avg       0.53      0.53      0.53    292069



In [34]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X_train, y_train)

# Predicting the Test set results
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.60      0.61    145813
           1       0.61      0.63      0.62    146256

    accuracy                           0.61    292069
   macro avg       0.61      0.61      0.61    292069
weighted avg       0.61      0.61      0.61    292069



In [35]:
from sklearn import metrics 
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

cnf_matrix

array([[87385, 58428],
       [54586, 91670]])

In [36]:
from sklearn.ensemble import RandomForestClassifier


# create a random forest classifier object
rf = RandomForestClassifier(n_estimators=1000, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7049053477089318


In [37]:
from sklearn import metrics 
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

cnf_matrix

array([[102236,  43577],
       [ 42611, 103645]])

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.70      0.70    145813
           1       0.70      0.71      0.71    146256

    accuracy                           0.70    292069
   macro avg       0.70      0.70      0.70    292069
weighted avg       0.70      0.70      0.70    292069



In [39]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=10, random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.582927321968439


In [40]:
from sklearn import metrics 
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

cnf_matrix

array([[88639, 57174],
       [64640, 81616]])

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5868202376835611


In [42]:
from sklearn import metrics 
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 
cnf_matrix

array([[82475, 63338],
       [57339, 88917]])

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58    145813
           1       0.58      0.61      0.60    146256

    accuracy                           0.59    292069
   macro avg       0.59      0.59      0.59    292069
weighted avg       0.59      0.59      0.59    292069

